# T81-558: Applications of Deep Neural Networks
**Class 8: Convolutional Neural Networks.**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

*Please note, this course file is under construction, a final version will be posted prior to the class meeting*

# Computer Vision

More to come

